# Tutorial 3 - System space coordinates
**Aim:** To recreate the double pendulum optimization model, but this time using coordinates in the system space instead of the joint space.

## Recap: what's the system space, again?
The system space is the set of $6n$ coordinates that you need to describe the position and orientation of $n$ links with respect to a fixed world frame.

<img src = "coordinate_systems.png" width = "400">

**Why model in the system space?** This is a valid question since you've just learned a generalized coordinate method that works for everything we're trying to do. So here are the pros and cons of the system space/ maximal approach as we've found them:

**PRO: It simplifies the equations of motion.** This is really the big one, but it's not a really clear advantage when you're working with a system as simple as the double pendulum. Here's the equation of motion for the second link, as derived with generalized coordinates:

$$1.0 In_{2} \ddot{\theta}_{1} + 1.0 In_{2} \ddot{\theta}_{2} + 0.5 \ddot{\theta}_{1} l_{1} l_{2} m_{2} \cos{\left (\theta_{2} \right )} + 0.25 \ddot{\theta}_{1} l_{2}^{2} m_{2} + 0.25 \ddot{\theta}_{2} l_{2}^{2} m_{2} + 0.5 \dot{\theta}_{1}^{2} l_{1} l_{2} m_{2} \sin{\left (\theta_{2} \right )} + 0.5 g l_{2} m_{2} \sin{\left (\theta_{1} + \theta_{2} \right )} - 1.0 \tau = 0$$

Not incredibly horrendous, but imagine what it might look like if there were four links in the system instead of just two... e.g. if it was the foot link, with a calf, thigh and torso above it. Because, in a generalized coordinate system, everything is defined relative to the link before, the equations can only get longer as you move down the chain. By the time you get to three or four links, you have something that's really unwieldly and difficult to debug by inspection. And let's not even get started on what happens in 3D.

By contrast, in the system space approach, the pose of each link is defined relative to the world frame, so the equations of motion stay consistently simple for all of them.

**CON: You need more constraints.** The major benefit of the generalized coordinate approach, and probably the reason why it's the most widely-applied option in robotics at present, is that all the connections between links just implicitly happen. While you still have some idea about the model being constructed from a number of discrete links, you're ultimately always treating it as a single body with no thought given to the forces and torques acting between the links to hold them together.

In the system space approach, we model the dynamics of the links as individual bodies, and explicitly include these connection forces and torques. This means, in addition to the equations of motion, you need to have a bunch of connection constraints telling the solver how the links need to interact in space i.e. which points on each link need to move together, which links need to point in the same direction, etc. (I'll demonstrate how to do this for a rotary joint now now).

**PRO: it's better for closed chains.** The maths of closed chains with traditional lagrange mechanics is so awful that even when we're modelling the rest of the system using generalized coordinates, we'll close chains using the connection constraint approach instead so we don't have to bother with it.

**CON: You still need access to the joint space coordinates, anyway.** The relative positions of the links are generally much more useful than the absolute positions: they goven the positional limits (e.g. how far a joint can move) and are what we typically think of when it comes to controlling the robot, so we'll need to include them - and maybe even the joint velocities, in some cases - in the model as auxiliary variables.

**PRO: System space solves faster (maybe).** This is actually the topic of a research paper we're working on right now. It's probably much of a muchness for simple systems, but for the more complex, high-DOF systems we typically work on (e.g. a four-legged animal with a flexible spine and tail) the annecdotal evidence points to it being much more tractable.

**TL;DR** System space modelling requires more constraints and variables, but the equations are much simpler and therefore easier to work with, and the resulting problems might be easier to solve (though we haven't proven that yet). The point of this tutorial isn't to tell you that it's necessarily better than using generalized coordinates, it's just to teach you how to do it and then you're free to decide which approach suits you and your specific problem better.

In [ ]:
%reset 
# clears variables in workspace

# import libraries
# Pyomo stuff
from pyomo.environ import*
from pyomo.opt import SolverFactory
from pyomo.opt import SolverStatus, TerminationCondition

# other
import numpy as np

from IPython.display import display #for pretty printing

# create the model
m = ConcreteModel()

# SETS-----------------------------------------------------------------------------------------------------------------------

N = 400 # how many points are in the trajectory 
#(you may have noticed it's a lot more than last time. Reason will be explained later)
m.N = RangeSet(N) # For defining ordered/numerical sets. Works like 'range' in python.
m.L = RangeSet(2)

# you can create sets with labels that aren't numeric:
DOFs = ['x','y','theta']
m.DOF = Set(initialize = DOFs) # the coordinates for each link

# PARAMETERS (same as last time) --------------------------------------------------------------------------------------------

m.g = Param(initialize = 9.81)
m.X0 = Param(initialize = 0.0) # position of fixed base
m.Y0 = Param(initialize = 2.0)
m.h = Param(initialize = 0.005) # time step

m.m = Param(m.L, initialize = 1.0) # mass of links
m.len = Param(m.L, initialize = 1.0) # length of links

def calculate_In(m, l): 
    return m.m[l]*m.len[l]**2/12 
m.In = Param(m.L, initialize = calculate_In) # moment of inertia

# VARIABLES -----------------------------------------------------------------------------------------------------------------

# system coordinates
m.X = Var(m.N, m.L, m.DOF) # position
m.dX = Var(m.N, m.L, m.DOF) # velocity
m.ddX = Var(m.N, m.L, m.DOF) # acceleration

# joint coordinates
m.th_joint = Var(m.N) # the only joint with a relative position different to the absolute angle is the second joint

# control torques
m.Tc = Var(m.N) # control torque at second joint

# CONSTRAINTS ---------------------------------------------------------------------------------------------------------------

# defining auxiliary variables
def def_th_joint(m,n):
    return m.th_joint[n] == m.X[n,2,'theta'] - m.X[n,1,'theta']
m.def_th_joint = Constraint(m.N, rule = def_th_joint)

# Integration constraints (now done on system coordinates)
def BwEuler_p(m,n,l,dof): # for positions
    if n > 1:
        return m.X[n,l,dof] == m.X[n-1,l,dof] + m.h*m.dX[n,l,dof]
    else:
        return Constraint.Skip #use this to leave out members of a set that the constraint doesn't apply to
m.integrate_p = Constraint(m.N, m.L, m.DOF, rule = BwEuler_p)

def BwEuler_v(m,n,l,dof): # for velocities
    if n > 1:
        return m.dX[n,l,dof] == m.dX[n-1,l,dof] + m.h*m.ddX[n-1,l,dof]
    else:
        return Constraint.Skip 
m.integrate_v = Constraint(m.N, m.L, m.DOF, rule = BwEuler_v)

# COST FUNCTION--------------------------------------------------------------------------------------------------------------

def CostFun(m):
    torque_sum = 0
    penalty_sum = 0
    for n in range(1,N+1):
        torque_sum += m.Tc[n]**2
    return torque_sum
m.Cost = Objective(rule = CostFun)

## Creating a rotary joint
As mentioned, the biggest downside of this approach is that you have to write explicit connection constraints to force links to interact with each other/ move together in space in ways conforming to the type of joint you want. And I do mean *force*, since part of the process of creating a joint is adding in constraint forces and torques (yup, these are more variables) to make the dynamics of these connections possible.

When you create a joint, what you're actually doing is restricting (or entirely removing) degrees of freedom from the relative motion between two links. The golden rule is:

**For every relative DOF restricted by a joint, a connection force must be added.**

You can also think of the control (aka *actuator*) forces/torques in the model as being part of the joints. Not every unrestricted DOF has to have a corresponding control action, though: some joints are unactuated and just move freely in response to whatever else is going on in the system.

Constraint and actuator forces must act on both links involved, in an equal and opposite fashion. 

Let's get to our example:

### The rotary joint
This model requires two joints:
1. to connect the first link to the fixed base point. Unactuated.
2. to connect the two links together. Actuated by the torque $\tau_c$

Both are rotary joints.

The first ingredients for a joint are the points of interaction: the points on each link that we'll be considering the relative motion to be between. For joint 1, the points are the fixed base $X_0, Y_0$ and the top of the first link. Joint 2 connects the bottom of the first link to the top of the second link. It's useful to create auxiliary variables to represent these points.

In a rotary joint, the points of interaction are not allowed any relative translational motion (they must have the same $x,y$ coordinates) but the links are allowed to rotate relative to each other. This means we need two constraint forces: $F_j = [F_j(x), F_j(y)]$. (We'll get to that when we write the equations of motion.) I never bound the magnitudes of constraint forces - the solver should be able to make them as big as they have to be to satisfy the connection constraints.

**Sidenote:** While the faff of having to specify all this extra stuff is definitely a drawback, access to the values of the constraint forces can actually be a huge advantage, in some very specific applications. For instance, seeking out a motion for rapid deceleration that minimizes say, the transverse force affecting the knee joint would be almost impossible without explicitly-modelled joints.

In [ ]:
# JOINTS

# joint variables
m.top = Var(m.N, m.L, m.DOF) # the top of each link
m.bottom = Var(m.N, m.L, m.DOF) # the top of each link

m.Fj = Var(m.N, m.L, m.DOF) # constraint forces. Fj[l] is the vector of forces acting at the top of the l'th joint.

# NOTE: you'll see we're creating these variables for all three DOF's, even though the joint only acts on x and y
# It's perfectly fine to use the whole set even when some elements will be redundant:
# variables that aren't involved in any constraints don't affect the solving process - they basically fall away.
# just be careful to make sure redundant DOF's are, in fact excluded (Constraint.Skip is your friend.)

# define joint variables
def def_top(m,n,l,dof):
    if dof == 'x':
        return m.top[n,l,dof] == m.X[n,l,dof] - 0.5*m.len[l]*sin(m.X[n,l,'theta']) 
    if dof == 'y':
        return m.top[n,l,dof] == m.X[n,l,dof] + 0.5*m.len[l]*cos(m.X[n,l,'theta']) 
    if dof == 'theta':
        return Constraint.Skip
m.def_top = Constraint(m.N, m.L, m.DOF, rule = def_top)

def def_bottom(m,n,l,dof):
    if dof == 'x':
        return m.bottom[n,l,dof] == m.X[n,l,dof] + 0.5*m.len[l]*sin(m.X[n,l,'theta']) 
    if dof == 'y':
        return m.bottom[n,l,dof] == m.X[n,l,dof] - 0.5*m.len[l]*cos(m.X[n,l,'theta'])
    if dof == 'theta':
        return Constraint.Skip
m.def_bottom = Constraint(m.N, m.L, m.DOF, rule = def_bottom)

# connection penalty

m.violation = Var(m.N, m.DOF) #small margin for violation in the top joint (more on that in a moment.)

for n in range(1,N):
    for dof in DOFs:
        m.violation[n,dof].setub(0.01)
        m.violation[n,dof].setlb(-0.01)

# connection constraints
def connect1(m,n,dof):
    if dof == 'x':
        return m.top[n,1,'x'] - m.X0 == m.violation[n,'x']
    if dof == 'y':
        return m.top[n,1,'y'] - m.Y0 == m.violation[n,'y']
    if dof == 'theta':
        return Constraint.Skip
m.connect1 = Constraint(m.N, m.DOF, rule = connect1)

def connect2(m,n,dof):
    if dof == 'theta':
        return Constraint.Skip
    else:
        return m.top[n,2,dof] - m.bottom[n,1,dof] == 0

m.connect2 = Constraint(m.N, m.DOF, rule = connect2)

## Equations of motion
The equations of motion are derived based on the following free-body diagram of the links:

<img src = "freebody.png" width = "400">

For the constraint and actuator forces, the convention I use is to assume positive direction on the lower (or child) link, and negative direction on the upper (or parent) link. It doesn't matter what convention you use, as long as you use one.

In [ ]:
def EOM1(m,n,dof):
    if dof == 'x':
        return m.m[1]*m.ddX[n,1,dof] == m.Fj[n,1,'x'] - m.Fj[n,2,'x']
    if dof == 'y':
        return m.m[1]*m.ddX[n,1,dof] == m.Fj[n,1,'y'] - m.Fj[n,2,'y'] - m.m[1]*m.g
    if dof == 'theta':
        # moments from the constraint forces
        mo_top = -m.Fj[n,1,'x']*0.5*m.len[1]*cos(m.X[n,1,'theta']) - m.Fj[n,1,'y']*0.5*m.len[1]*sin(m.X[n,1,'theta'])
        mo_bottom = -m.Fj[n,2,'x']*0.5*m.len[1]*cos(m.X[n,1,'theta']) - m.Fj[n,2,'y']*0.5*m.len[1]*sin(m.X[n,1,'theta'])
        return m.In[1]*m.ddX[n,1,dof] == mo_top + mo_bottom - m.Tc[n]
m.EOM1 = Constraint(m.N, m.DOF, rule = EOM1)

def EOM2(m,n,dof):
    if dof == 'x':
        return m.m[2]*m.ddX[n,2,dof] == m.Fj[n,2,'x']
    if dof == 'y':
        return m.m[2]*m.ddX[n,2,dof] == m.Fj[n,2,'y'] - m.m[2]*m.g
    if dof == 'theta':
        mo_top = -m.Fj[n,2,'x']*0.5*m.len[2]*cos(m.X[n,2,'theta']) - m.Fj[n,2,'y']*0.5*m.len[2]*sin(m.X[n,2,'theta'])
        return m.In[2]*m.ddX[n,2,dof] == mo_top + m.Tc[n]
m.EOM2 = Constraint(m.N, m.DOF, rule = EOM2)

## Guesses in the system space
One thing you'll have noticed is that system space relies much more heavily on aux variables than joint space does, and you might be wondering what the implications are for initializing your model.

In my mind, there are two broad approaches you could take to this step:

**Option 1: make every node as feasible as possible.** Here, you first initialize the poses using your method of choice (randomly or constant or close to an expected trajectory, depending on your goals) and then attempt to calculate intial values for all the auxiliary variables based on these positions. 

(You may also have to initialize some other fundamental variables, such as the forces, to have all the ingredients you need to calculate the auxiliaries. As with the poses, you can do so in whichever way best suits your application.)

Note: even though my system space variables are technically the *decision* ones, I randomize my positions using the joint space variables and then calculate the corresponding system space values. Why? Doing it this way lets me satisfy the connection constraints: if I randomized the system space coordinates, I'd end up with a chaotic storm of dismembered limbs flying around all over the place, while at least with random joint positions I can still get a sequence of coherent poses.

**Option 2: {shrugs}** Option 1 is obviously the high-effort version and, to be honest, I'm not sure how *worth it* it really is. Anecdotal evidence suggests it might start the solver off in a better (i.e. more feasible) place, so I do make the effort when I'm rolling out a model for a series of experiments, but the gains for simple systems like the pendulum aren't likely to be anything worth the trouble.

So you can just initialize your poses however you see fit, and then either assign a smallish constant value to the auxiliaries, or leave them unspecified.

For this tut, I'm going to lean much more on the lazy side: randomized joint positions, system space coordinates calculated from those, everything else constant or ignored.

In [ ]:
# INITIAL GUESS

for n in range(1,N+1):
    m.Tc[n].value = 1
    
    #m.X[n,1,'theta'].value = 0.1
    #m.th_joint[n].value = 0.1
    
    #m.X[n,1,'theta'].value = np.random.uniform(-np.pi,np.pi)
    #m.th_joint[n].value = np.random.uniform(-np.pi,np.pi)
    
    for l in range(1,3):
        for dof in DOFs:
            m.dX[n,l,dof].value = 1
            m.ddX[n,l,dof].value = 10
            
    #calculate positions:
    m.top[n,1,'x'].value = m.X0.value
    m.top[n,1,'y'].value = m.Y0.value
    
    m.X[n,1,'x'].value = m.X0.value + 0.5*m.len[1]*np.sin(m.X[n,1,'theta'].value)
    m.X[n,1,'y'].value = m.Y0.value - 0.5*m.len[1]*np.cos(m.X[n,1,'theta'].value)
    
    m.bottom[n,1,'x'].value = m.X0.value + m.len[1]*np.sin(m.X[n,1,'theta'].value)
    m.bottom[n,1,'y'].value = m.Y0.value - m.len[1]*np.cos(m.X[n,1,'theta'].value)
    
    m.X[n,2,'theta'].value = m.th_joint[n].value + m.X[n,1,'theta'].value
    
    m.top[n,2,'x'].value = m.bottom[n,1,'x'].value
    m.top[n,2,'y'].value = m.bottom[n,1,'y'].value
    
    m.X[n,2,'x'].value = m.top[n,2,'x'].value + 0.5*m.len[2]*np.sin(m.X[n,2,'theta'].value)
    m.X[n,2,'y'].value = m.top[n,2,'y'].value - 0.5*m.len[2]*np.cos(m.X[n,2,'theta'].value)
    
    m.bottom[n,2,'x'].value = m.X0.value + m.len[2]*np.sin(m.X[n,2,'theta'].value)
    m.bottom[n,2,'y'].value = m.Y0.value - m.len[2]*np.cos(m.X[n,2,'theta'].value)
        

In [ ]:
# BOUNDS

for n in range(1,N):
    m.Tc[n].setub(50)
    m.Tc[n].setlb(-50)

# initial condition
m.X[1,1,'theta'].value = 0
m.X[1,2,'theta'].value = 0
m.X[1,1,'theta'].fixed = True
m.X[1,2,'theta'].fixed = True

for dof in DOFs:
    for l in range(1,3):
        m.dX[1,l,dof].value = 0
        m.dX[1,l,dof].fixed = True
        
# final condition
m.X[N,1,'theta'].value = np.pi
m.X[N,2,'theta'].value = np.pi
m.X[N,1,'theta'].fixed = True
m.X[N,2,'theta'].fixed = True

for dof in DOFs:
    for l in range(1,3):
        m.dX[N,l,dof].value = 0
        m.dX[N,l,dof].fixed = True

## The fixed base problem
This is actually a really terrible way to model a fixed-base system, but I did it this way because it allows me to introduce the main villain in our story: **hard contacts**. (*dun dun duuuuunnnn*)

These occur in two primary places in the typical systems we'll be modelling:
1. contacts between the foot and the ground (or perhaps between two objects, such as a foot and a skateboard ;) )
2. the endpoints of a joint's range of motion

When we model the double pendulum the way we have here, we're essentially trying to impose the fixed-base condition by means of a hard contact between the top of the pendulum and some invisible structure that it's mounted on. (If you imagine the pedulum swinging from a bolt that's stuck into a wall somewhere, the constraint force $F_j$ would be the normal force exerted on the bolt.)

Thinking about it from the point-of-view of humans living in a smooth, continuous-time world, this seems to make physical sense, but we are working in the discrete-time world of trajectory optimization and things are different here. 

If you're interested in seeing what's going on in terms of the way we've formulated our model mathematically, I've done the working in an Appendix (it's a separate notebook), but I'll try to explain the problem in a less mathy, more intuitive way here.

Imagine that, instead of being stuck into a single point in the wall, the bolt from which the pendulum hangs is mounted in a slot, so it has a little bit of leeway to slide left and right. A vigorous swing pulls the top of the link all the way to the left, at which point it hits the end of the slot and is acted upon by a rightward rebound force.

Consider the node $i$ representing the instant it hits the edge of the slot (the timing of this is a whole issue in itself that I'll talk about more in the next tut, which deals with contacts in detail): in that moment, $x$ is at the minimum value it can have and $\dot{x}$ is some nonzero leftward velocity. Since it can't move any further to the left, we know that at the next node $i + 1$, $x$ has to either have the same value or have displaced to the right. This means that the rebound force acting at $i$ needs to be large enough to nullify or reverse $\dot{x}$ over a single timestep.

Let's go with the case where the velocity changes direction. In fact, we almost certainly have to. Why? Remember: optimization world. Here, the force doesn't actually affect the end of the link directly: it acts on the $x$ position of the link's COM, and creates a moment that affects $\theta$, and together these actions cause some change in the position of the top-end. So it's not actually all that simple to choose the exact force that perfectly stops the end in its tracks. (This is mostly what that appendix deals with.) ...Anyway, the link is now moving to the right. Let's say the slot is narrow enough that it immediately hits the rightmost boundary after the next timestep, and the whole process repeats itself but in the opposite direction. *Again and again and again...* 

Assuming you've done a bit of control by now, I'm sure you'll recognize the ensuing behaviour as that of a horribly underdamped system: overshoot after overshoot, with the force 'ringing' rapidly in opposite directions. (You'll see this oscillation clearly when the force is plotted, and it's the reason for the ghastly shiver you'll notice in the animation.)

**In summary**, the problem is caused by two clashing non-idealnesses:
1. The constraint forces don't affect the link's endpoints directly, so there's always some error between the force applied and the force required to stop the endpoint.
2. The system has to remain in a particular state (i.e. acted on by the error force) for a whole timestep. So error force creates error acceleration, creates error velocity, creates error displacement... and so on in a miserable cycle of overshoot.

### Solving this problem the crappy way

Considering those limitations, you can see that the only ways to make the problem solvable for the case of a single fixed point (i.e. an infinitesimally small slot) are either (A) to make the timestep so small that the error displacements fall within the tiny ($1e^{-6}$) tolerance of the solver, or (B) if you don't want to spend the rest of your life waiting for a gazillion-node problem to converge, you can allow some small violation of the fixed position constraint.

You'll see in the code that I did a bit of both: I reduced the timestep to $frac{1}{4}$ what it was before (meaning I had to use $4 \times$ the number of nodes to represent the same time) and created an auxiliary violation variable I could use to bound the amount of error.

### Solving this problem the right way

So how should you deal with a fixed point joint using system space coordinates? Simple: *don't*. My workaround is to model the first link the same way we did in previous tutorials i.e. its position is represented only by a $\theta$ coordinate, so the connection to the dynamics at the fixed point become implicit. 

Think of it this way: even though the link's COM can translate, the link itself is only capable of rotational motion. Because the top is fixed, we only need to know $\theta$ to completely specify its position. The $x$ and $y$ coordinates don't represent degrees of freedom the link actually has, so we may as well discard them. 

(There is a short sequel to this tutorial where you can see how this is done.)

In [ ]:
# solving
#opt = SolverFactory('ipopt') # standard issue, garden variety ipopt

opt = SolverFactory('ipopt',executable = 'C:/cygwin64/home/Stacey/CoinIpopt/build/bin/ipopt.exe')
opt.options["linear_solver"] = 'ma86'

# solver options
opt.options["print_level"] = 5 # prints a log with each iteration (you want to this - it's the only way to see progress.)
opt.options["max_iter"] = 30000 # maximum number of iterations
opt.options["max_cpu_time"] = 600 # maximum cpu time in seconds (you'll need it...)
opt.options["Tol"] = 1e-6 # the tolerance for feasibility. Considers constraints satisfied when they're within this margin.
    
results = opt.solve(m, tee = True) 

#WARNING: takes ages

In [ ]:
# For debugging:
print(results.solver.status) # tells you if the solver had any errors/ warnings
print(results.solver.termination_condition) # tells you if the solution was (locally) optimal, feasible, or neither.

#m.pprint() 

In [ ]:
#animate it
import matplotlib.pyplot as plt
import matplotlib.animation as ani
from IPython.display import HTML
%matplotlib inline

fig1, ax1 = plt.subplots(1,1) #create axes

def plot_pendulum(i,m,ax): #update function for animation
    ax.clear()
    ax.set_xlim([-2,2])
    ax.set_ylim([0,4])
    
    #plot link 1
    L1topx = m.top[i,1,'x'].value
    L1topy = m.top[i,1,'y'].value
    L1bottomx = m.bottom[i,1,'x'].value
    L1bottomy = m.bottom[i,1,'y'].value  
    ax.plot([L1topx,L1bottomx],[L1topy,L1bottomy],color='xkcd:black')
    
    
    #plot link 2
    L2bottomx = m.bottom[i,2,'x'].value
    L2bottomy = m.bottom[i,2,'y'].value
    ax.plot([L1bottomx,L2bottomx],[L1bottomy,L2bottomy],color='xkcd:black')
    
update = lambda i: plot_pendulum(i,m,ax1) #lambdify update function

animate = ani.FuncAnimation(fig1,update,range(1,N+1),interval = 50,repeat=True)

HTML(animate.to_html5_video()) #you need to convert the animation to HTML5 to embed it in the notebook


In [ ]:
# plot the constraint force at the top joint (so you can see how gross it is)
Fj = []
for n in range(1,N+1):
    Fj.append(m.Fj[n,1,'x'].value)
plt.plot(range(len(Fj)),Fj)